# Test debris_tracking

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from importlib import reload
import shapely
from shapely import plotting as shapely_plotting
from clawpack.visclaw import animation_tools
from IPython.display import display, HTML


In [ ]:
import debris_tracking
reload(debris_tracking) # for debugging

In [ ]:
# equilateral triangle:

debris = debris_tracking.DebrisObject()
debris.L = [1,1]
debris.phi = [2*pi/3, 2*pi/3]
debris.z0 = [1,2,0]

# plot triangle with bottom edge flat:
xc,yc = debris.get_corners(close_poly=True)  # uses z = debris.z0
plot(xc, yc, 'b')
grid(True)
gca().set_aspect(1)

# same triangle, rotated by 45 degrees:
xc,yc = debris.get_corners(z=(1,2,pi/4), close_poly=True)
plot(xc, yc, 'r')
grid(True)
gca().set_aspect(1)


In [ ]:
def plot_quiver(extent, dx, dy, u, v):
    xvel = arange(extent[0], extent[1]+dx/2, dx)
    yvel = arange(extent[2], extent[3]+dy/2, dy)
    Xvel,Yvel = meshgrid(xvel,yvel,indexing='xy')
    Uvel = u(Xvel,Yvel,0)
    Vvel = v(Xvel,Yvel,0)
    smax = sqrt(Uvel**2 + Vvel**2).max()
    #print(f'Scaling by 1.1*smax/dx = {1.1*smax/dx}')
    quiver(Xvel,Yvel,Uvel,Vvel,scale=1.1*smax/dx,scale_units='xy',color='gray',width=0.003)

In [ ]:
# shear flow:
u = lambda x,y,t: y
v = lambda x,y,t: zeros(x.shape)
plot_quiver([-1,8,-1,3], 1, 0.5, u, v)


In [ ]:
def make_anim(debris_path_list, extent, figsize=(6,6), obst_list=[], domain=None, u=None, v=None):
    figs = [] # to accumulate frames of animation
    for n in range(nsteps+1):
        fig = figure(figsize=figsize)

        if (u is not None) and (v is not None):
            # quiver plot of velocity field:
            x1,x2,y1,y2 = extent
            if domain is not None:
                x1 = max(x1,domain[0])
                x2 = min(x2,domain[1])
                y1 = max(y1,domain[2])
                y2 = min(y2,domain[3])
            dx = (x2-x1)/20
            dy = (y2-y1)/20
            plot_quiver([x1,x2,y1,y2], dx, dy, u, v)

        # plot any obstacles:
        for obst in obst_list:
            shapely.plotting.plot_polygon(obst['polygon'], add_points=False,
                                          color='blue', alpha=0.5)

        if domain is not None:
            # plot any walls:
            x1,x2,y1,y2 = extent
            x1wall,x2wall,y1wall,y2wall = domain
            plot([x1wall,x1wall], [max(y1wall,y1),min(y2wall,y2)], 'g')
            plot([x2wall,x2wall], [max(y1wall,y1),min(y2wall,y2)], 'g')
            plot([max(x1wall,x1),min(x2wall,x2)], [y1wall,y1wall], 'g')
            plot([max(x1wall,x1),min(x2wall,x2)], [y2wall,y2wall], 'g')


        # plot debris paths:
        for debris_path in debris_path_list:
            debris = debris_path.debris
            t_n = debris_path.times[n]
            z_n = debris_path.z_path[n]
            xc,yc = debris.get_corners(z_n, close_poly=True)
            plot(xc, yc, 'b')
            
        #grid(True)
        axis(extent)
        title(f'Time {t_n} seconds')
        gca().set_aspect(1)
        figs.append(fig)
        close(fig)

    images = animation_tools.make_images(figs)
    anim = animation_tools.animate_images(images, figsize=figsize)
    return anim

In [ ]:
debris = debris_tracking.DebrisObject()
debris.L = [1,1,1]
debris.phi = [pi/2, pi/2, pi/2]
debris.z0 = [0,0.5,0]

debris.advect = True
debris.rho = 900.

debris_list = [debris]
z0_list = [db.z0 for db in debris_list]


# shear flow:
u = lambda x,y,t: y
v = lambda x,y,t: zeros(x.shape)
h = lambda x,y,t: 10.




# obstacles:

obst = debris_tracking.make_rectangular_obstacle(2,3,0.5,1)
obst_list = [obst]
obst_list = []  # no obstacles

# walls:

#domain = None  # no physical domain walls
xwall = inf
ywall = 0.5
domain = [-inf, xwall, ywall,inf]

# time stepping parameters:

t0 = 0.
nsteps = 10
dt = 0.5

debris_path_list = debris_tracking.make_debris_path_list(debris_list, z0_list,
                                                         obst_list, domain,
                                                         t0,dt,nsteps,h,u,v)


debris_path = debris_path_list[0]  # only one debris object

# make plots:

if 0:
    # plot several times on one static plot
    figure(figsize=(8,8))
    plot_quiver([-1,8,-1,3], 1, 0.5, u, v)
    c = 4*['k','r','b','g']
    for n in range(nsteps+1):
        t_n = debris_path.times[n]
        z_n = debris_path.z_path[n]
        xc,yc = debris.get_corners(z_n, close_poly=True)
        plot(xc, yc, color=c[n])

if 0:
    # make animation
    figs = [] # to accumulate frames of animation
    for n in range(nsteps+1):
        fig = figure(figsize=(6,4))

        # quiver plot of velocity field:
        plot_quiver([-1,8,-1,3], 1, 0.5, u, v)

        # plot any obstacles:
        for obst in obst_list:
            shapely.plotting.plot_polygon(obst['polygon'], add_points=False,
                                          color='blue', alpha=0.5)

        # plot debris path:
        t_n = debris_path.times[n]
        z_n = debris_path.z_path[n]
        xc,yc = debris.get_corners(z_n, close_poly=True)
        plot(xc, yc, 'b')
        title(f'Time {t_n} seconds')
        #grid(True)
        plot([0,8], [ywall,ywall], 'r')
        plot([xwall,xwall], [0,4], 'r')
        xlim(0,8)
        ylim(0,4)
        gca().set_aspect(1)
        figs.append(fig)
        close(fig)

    images = animation_tools.make_images(figs)
    anim = animation_tools.animate_images(images, figsize=(6,4))


print(f'domain = {domain}')
anim = make_anim(debris_path_list, [0,8,0,4], (6,4), obst_list, domain, u, v)
HTML(anim.to_jshtml())


In [ ]:
# Square debris:
debris = debris_tracking.DebrisObject()
debris.L = [1,1,1]
debris.phi = [pi/2, pi/2, pi/2]
debris.z0 = [0,0.5,0]

debris.advect = True
debris.rho = 900.

debris_list = [debris]

# Triangular debris:
debris = debris_tracking.DebrisObject()
debris.L = [1,1]
debris.phi = [2*pi/3, 2*pi/3]
debris.z0 = [-1.3,1.1,0]

debris.advect = True
debris.rho = 900.
debris_list.append(debris)

z0_list = [db.z0 for db in debris_list]


# shear flow:
u = lambda x,y,t: y
v = lambda x,y,t: zeros(x.shape)
h = lambda x,y,t: 10.


t0 = 0.
nsteps = 20
dt = 0.25

obst = debris_tracking.make_rectangular_obstacle(2,3,0.5,1)
obst_list = [obst]
#obst_list = []  # no obstacles
#domain = None  # no physical domain walls
xwall = inf
ywall = 0.5
domain = [-inf, xwall, ywall,inf]

debris_path_list = debris_tracking.make_debris_path_list(debris_list, z0_list,
                                                         obst_list, domain,
                                                         t0,dt,nsteps,h,u,v)


debris_path = debris_path_list[0]



if 1:
    # plot all times on same figure:
    extent = [-2,8,0,4]
    figure(figsize=(8,6))
    plot_quiver([-1,8,-1,3], 1, 0.5, u, v)

    for obst in obst_list:
        shapely_plotting.plot_polygon(obst['polygon'], add_points=False,
                                      color='blue', alpha=0.5)
    

    c = 10*['k','r','b','g']
    for n in range(nsteps+1):
        for debris_path in debris_path_list:
            debris = debris_path.debris
            t_n = debris_path.times[n]
            z_n = debris_path.z_path[n]
            xc,yc = debris.get_corners(z_n, close_poly=True)
            plot(xc, yc, color=c[n], label='t = %.1f' % t_n)
    
    #legend()
    axis(extent)
    gca().set_aspect(1)
    grid(True)
    plot([0,8], [ywall,ywall], 'r')
    plot([xwall,xwall], [0,4], 'r')
    title('All frames')


# make animation:
extent = [-2,8,0,4]
figsize = (6,4)
anim = make_anim(debris_path_list, extent, figsize, obst_list, domain, u, v)
HTML(anim.to_jshtml())

## Swirling flow

In [ ]:
figure(figsize=(8,8))
u = lambda x,y,t: -2*sin(pi*y)*cos(pi*y)*sin(pi*x)**2
v = lambda x,y,t: 2*sin(pi*x)*cos(pi*x)*sin(pi*y)**2
plot_quiver([0,1,0,1], 0.05, 0.05, u, v)
axis('scaled')
xlim(0,1)
ylim(0,1)

In [ ]:
debris = debris_tracking.DebrisObject()
debris.L = [0.3,0.3,0.3]
debris.phi = [pi/2, pi/2, pi/2]
debris.z0 = [0.6,0.2,0]

debris.advect = True
debris.rho = 900.
debris_list = [debris]


# Triangular debris:
debris = debris_tracking.DebrisObject()
debris.L = [0.4,0.4]
debris.phi = [2*pi/3, 2*pi/3]
debris.z0 = [0.5, 0.6, 0]

debris.advect = True
debris.rho = 900.
debris_list.append(debris)

z0_list = [db.z0 for db in debris_list]

# swirling flow:
u = lambda x,y,t: -2*sin(pi*y)*cos(pi*y)*sin(pi*x)**2
v = lambda x,y,t: 2*sin(pi*x)*cos(pi*x)*sin(pi*y)**2
h = lambda x,y,t: 10.

t0 = 0.
nsteps = 30
dt = 0.25

#obst = debris_tracking.make_rectangular_obstacle(2,3,0.5,1)
#obst_list = [obst]
obst_list = []

#domain = None  # no physical domain walls
domain = [0,1,0,1]

debris_path_list = debris_tracking.make_debris_path_list(debris_list, z0_list,
                                                         obst_list, domain,
                                                         t0,dt,nsteps,h,u,v)



# make animation:
extent = [-0.1,1.1,-0.1,1.1]
figsize = (6,6)
anim = make_anim(debris_path_list, extent, figsize, obst_list, domain, u, v)
HTML(anim.to_jshtml())